In [ ]:
!pip install pycaret tqdm --quiet
!python3 -m spacy download en --quiet

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/lucas/anaconda3/envs/quant/lib/python3.7/site-packages/en_core_web_sm -->
/home/lucas/anaconda3/envs/quant/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import pandas as pd # para abrir e visualizar seus dados
import matplotlib.pyplot as plt # para a análise de dados
import seaborn as sns # para a análise também
import numpy as np # porque é sempre importar numpy, né?

from tqdm import tqdm
import nltk # para pré-processar o texto
nltk.download('stopwords') # nltk exige que a gente baixe as stopwords
from nltk.probability import FreqDist # para checarmos frequência de palavras
from nltk.tokenize import word_tokenize # para tokenizar
nltk.download('punkt') # nltk exige que a gente baixe o 'punkt'

import re # para pré-processar o texto

import spacy # para pré-processar o texto
spc = spacy.load('en') # spacy exige que a gente carregue a língua que vamos usar

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from pycaret.classification import *
from pycaret.utils import enable_colab
import pycaret

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
enable_colab()

Colab mode enabled.


In [ ]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df = df.drop('id',axis=1)
df = df.drop('author',axis=1)
df = df.drop('title',axis=1)

In [ ]:
df['text'] = df['text'].astype(str)

In [ ]:
english_stop_words = nltk.corpus.stopwords.words('english')
stop_words = set(english_stop_words)

In [ ]:
def remove_stop_words(texto, stop_words):
    
    tokens =  re.findall(r'\b[A-zÀ-úü]+\b', texto)

    sem_stopwords = [token for token in tokens if token not in stop_words]
    
    return " ".join(sem_stopwords)

In [ ]:
def lematiza_frase(doc):
    lemma_list = [str(tok.lemma_).lower() for tok in doc
                  if tok.is_alpha and tok.text.lower() not in stop_words] 
    return lemma_list

def lematiza_textos(textos):
    tokens_processados = []
    
    for documentos in tqdm(spc.pipe(textos, batch_size=200, n_process=8, disable=["parser", "ner"])):
        tokens_processados.append(lematiza_frase(documentos))
        
    textos_processados = [" ".join(tokens) for tokens in tokens_processados]
        
    return textos_processados

In [ ]:
df['text'] = df['text'].apply(lambda texto: texto.lower())
df['text'] = df['text'].apply(lambda texto: remove_stop_words(texto, stop_words))

In [ ]:
df['text'] = lematiza_textos(df['text'])

20800it [04:29, 77.19it/s] 


In [ ]:
df.head()

,text,label
0,house dem aide even see comey letter jason cha...,1
1,ever get feel life circle roundabout rather he...,0
2,truth may get fire october tension intelligenc...,1
3,video civilian kill single -pron- airstrike id...,1
4,print iranian woman sentence six year prison i...,1


In [ ]:
# instanciado o countvec
vectorizer = CountVectorizer(max_features=100) 

# definindo a feature de texto como vector_text
textos = df['text']

# dando um fit_transform na feature de texto e colocando em x
vetor_textos = vectorizer.fit_transform(textos).toarray()

In [ ]:
frequencias = pd.DataFrame(vetor_textos, columns = vectorizer.get_feature_names())

In [ ]:
frequencias.head()

,accord,also,american,another,back,become,call,campaign,case,change,...,way,week,well,white,woman,work,world,would,write,year
0,3,2,1,0,0,1,0,0,4,0,...,1,1,1,0,0,1,3,0,0,2
1,0,0,2,0,1,1,1,0,0,0,...,3,0,1,1,4,0,0,0,0,1
2,0,1,2,1,1,1,0,1,1,1,...,4,0,5,1,0,2,1,1,1,1
3,0,0,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4,0,0,0,2,2


In [ ]:
frequencias['e_fake_news'] = df['label']

In [ ]:
py = setup(data = frequencias, target = 'e_fake_news', train_size=0.05)

,Description,Value
0,session_id,2373
1,Target,e_fake_news
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(20800, 101)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,91
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8096,0.8867,0.8220,0.8089,0.8147,0.6191,0.6204,0.3810
lightgbm,Light Gradient Boosting Machine,0.7962,0.8791,0.8069,0.7972,0.8010,0.5922,0.5938,0.0830
ada,Ada Boost Classifier,0.7894,0.8599,0.7975,0.7927,0.7940,0.5788,0.5806,0.1180
rf,Random Forest Classifier,0.7779,0.8603,0.7407,0.8091,0.7722,0.5563,0.5599,0.3250
lr,Logistic Regression,0.7683,0.8609,0.7860,0.7660,0.7752,0.5362,0.5374,0.5860
svm,SVM - Linear Kernel,0.7654,0.0000,0.7653,0.7775,0.7679,0.5307,0.5356,0.0330
et,Extra Trees Classifier,0.7548,0.8443,0.6953,0.7997,0.7416,0.5106,0.5174,0.3050
knn,K Neighbors Classifier,0.7500,0.8167,0.8466,0.7178,0.7756,0.4984,0.5090,0.1060
dt,Decision Tree Classifier,0.7163,0.7028,0.7105,0.7292,0.7174,0.4328,0.4356,0.0420
ridge,Ridge Classifier,0.7106,0.0000,0.7655,0.6973,0.7283,0.4201,0.4244,0.0490
